In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import indicators
from db import timscale_setup



In [ ]:
import PyAlgoTrade
import pandas_datareader
import pytz
import pyfolio as pf
from collections import OrderedDict

#quandl.ApiConfig.api_key = "1dkdmifTyDZVCKaXnKTD"


In [2]:
print("Fetching Data")
data = timscale_setup.get_full_table("BANKNIFTY_F1")
data.describe()
data.head()

Fetching Data


,datetime,internalname,open,high,low,close,volume,unknown,expirydate,exchange
0,2020-01-01 09:16:00,BANKNIFTY JAN FUT,32417.30,32465.15,32353.90,32456.05,36240,1304980,2020-01-30 23:59:00,NSE
1,2020-01-01 09:17:00,BANKNIFTY JAN FUT,32457.00,32466.00,32444.55,32444.55,18780,1304980,2020-01-30 23:59:00,NSE
2,2020-01-01 09:18:00,BANKNIFTY JAN FUT,32446.40,32452.00,32445.00,32445.00,14900,1316960,2020-01-30 23:59:00,NSE
3,2020-01-01 09:19:00,BANKNIFTY JAN FUT,32445.00,32463.65,32435.00,32463.65,18120,1316960,2020-01-30 23:59:00,NSE
4,2020-01-01 09:20:00,BANKNIFTY JAN FUT,32460.75,32474.00,32456.90,32465.00,11540,1316960,2020-01-30 23:59:00,NSE


In [17]:
datetime_index = pd.DatetimeIndex(pd.to_datetime(data["datetime"]).values)
data_backtesting = data.set_index(datetime_index)
data_backtesting.drop(["datetime", "internalname", "unknown", "expirydate", "exchange"], axis=1, inplace=True)
data_backtesting.rename(columns = {'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'}, inplace = True)

data_backtesting.head()

,Open,High,Low,Close,Volume
2020-01-01 09:16:00,32417.30,32465.15,32353.90,32456.05,36240
2020-01-01 09:17:00,32457.00,32466.00,32444.55,32444.55,18780
2020-01-01 09:18:00,32446.40,32452.00,32445.00,32445.00,14900
2020-01-01 09:19:00,32445.00,32463.65,32435.00,32463.65,18120
2020-01-01 09:20:00,32460.75,32474.00,32456.90,32465.00,11540


In [19]:
#from backtesting.test import SMA
from backtesting import Strategy
from backtesting.lib import crossover


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 20
    n2 = 50
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()



In [ ]:
class SmaCrossSignalTrail(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)


In [21]:
from backtesting.test import GOOG

GOOG.head()

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900


In [4]:
from backtesting import Backtest


bt = Backtest(GOOG, SmaCross, cash=10000, commission=.002)
stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     94.2737
Equity Final [$]                      68935.1
Equity Peak [$]                       68991.2
Return [%]                            589.351
Buy & Hold Return [%]                 703.458
Max. Drawdown [%]                    -33.0822
Avg. Drawdown [%]                    -5.58151
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   93
Win Rate [%]                          53.7634
Best Trade [%]                        57.1193
Worst Trade [%]                      -16.6299
Avg. Trade [%]                        1.95909
Max. Trade Duration         121 days 00:00:00
Avg. Trade Duration          32 days 00:00:00
Profit Factor                         2.12576
Expectancy [%]                        6.91487
SQN                               

In [26]:
bt_banknifty = Backtest(data_backtesting, SmaCross, cash=100000, commission=.002)
banknifty_stats = bt_banknifty.run()

print(banknifty_stats)

Start                     2020-01-01 09:16:00
End                       2020-07-31 18:39:00
Duration                    212 days 09:23:00
Exposure Time [%]                     97.1008
Equity Final [$]                      21501.9
Equity Peak [$]                        100000
Return [%]                           -78.4981
Buy & Hold Return [%]                 33.1665
Max. Drawdown [%]                    -78.5633
Avg. Drawdown [%]                    -78.5633
Max. Drawdown Duration      212 days 06:32:00
Avg. Drawdown Duration      212 days 06:32:00
# Trades                                 1199
Win Rate [%]                           22.769
Best Trade [%]                        8.39018
Worst Trade [%]                       -8.7021
Avg. Trade [%]                      -0.164492
Max. Trade Duration           3 days 19:10:00
Avg. Trade Duration           0 days 04:11:00
Profit Factor                        0.614501
Expectancy [%]                       0.660663
SQN                               

In [27]:

bt_banknifty.plot()

In [24]:
bt.plot()

In [28]:
%%time

stats = bt_banknifty.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
print(stats)

KeyboardInterrupt: 

In [10]:
print(stats._strategy)
stats.tail()

SmaCross(n1=10,n2=15)


Sortino Ratio                                             0.392356
Calmar Ratio                                             0.0345735
_strategy                                    SmaCross(n1=10,n2=15)
_equity_curve                      Equity  DrawdownPct Drawdown...
_trades               Size  EntryBar  ExitBar  EntryPrice  Exit...
dtype: object

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [ ]:
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
benchmark_period_return = perf['benchmark_period_return']

# Convert benchmark returns to daily returns
#daily_returns = (1 + benchmark_period_return) / (1 + benchmark_period_return.shift()) - 1
daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

# Create tear sheet
#pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=daily_benchmark_returns)

## Daily returns from cummulative returns


# We need to be able to calulate the daily returns from the cumulative returns
daily_returns = pd.Series([0.5, -0.5, 0.5, -0.5])
cumulative_returns = pd.Series([0.5, -0.25, 0.125, 0.5625])

# Two different formulas to calculate daily returns
print((1 + cumulative_returns) / (1 + cumulative_returns.shift()) -1)
print((np.exp(np.log(cumulative_returns + 1).diff()) - 1))

# Recreate daily returns manually for example purposes
print(daily_returns.head(1))
print((1 - 0.25) / (1.5) - 1)
print((1 + 0.125) / (1 - 0.25) - 1)
print((1 + 0.5625) / (1 + 0.125 ) - 1)
